# TSMC: The SPIN Model Checker
* https://spinroot.com/spin/Doc/Book_extras/
  * introduction: 1-5
  * foundations: 6-10
  * practice: 11-15
  * reference material: 16-19
  * appendices: A-D 

- version: 4.0.7 - P.738


topics:
* PROMELA: 3, 7, 16, 17
* 描述正确性属性: 4
  * 时间线编辑器: 13
* 使用SPIN做系统验证: 11, 14
* XSPIN图形用户界面: 12
* 在SPIN模型中内嵌C代码: 10, 17
* 自动机和逻辑: 6

In [3]:
# 工作目录
%cd /mnt/d/GoogleDrive/wiki/jupyter-notebooks/Modeling/SPIN/TSMC

/mnt/d/GoogleDrive/wiki/jupyter-notebooks/Modeling/SPIN/TSMC


# 1. Finding Bugs in Concurrent Systems

- 圆阻塞(circular blocking)
- 致命拥抱(deadly embrace)
- 不匹配的假设(mismatched assumption): 组件层的假设在系统层不适用.

可重现的测试(reproducible tests) 无法实现的原因:
- 受限的可观测性: 进程执行的细节
- 受限的可控制性: 进程调度器确定的进程交错(interleaving)

需要不同类型的验证方法!

# 2. Building Verification Models

Examples:
- Hello World: hello.pml
- Producers and Consumers: prodcons.pml
- Mutual Exlusion
- Message Passing

In [2]:
# 模拟运行
!spin hello.pml

          hello world from init
      hello world
2 processes created


In [ ]:
# 不确定性(non-determinism)
#  多个guard条件同时为true, SPIN不确定的选择一个执行
#  进程调度的不确定性: 一个执行中多个进程有可执行的语句并可进展, PROMELA的语义声明可以选择任意一个进程执行.

In [3]:
# 模拟运行最多14步
!spin -u14 prodcons.pml

      Produce
          Consume
      Produce
          Consume
-------------
depth-limit (-u14 steps) reached
#processes: 2
		turn = C
 14:	proc  1 (consumer:1) prodcons.pml:37 (state 3)
 14:	proc  0 (producer:1) prodcons.pml:8 (state 4)
2 processes created


In [5]:
!spin -u100 prodcons2.pml

          P1
              C2
          P1
                  C3
          P1
              C2
      P0
              C2
      P0
              C2
          P1
              C2
          P1
-------------
depth-limit (-u100 steps) reached
#processes: 4
		turn = N
		who = 1
100:	proc  3 (consumer:1) prodcons2.pml:26 (state 12)
100:	proc  2 (consumer:1) prodcons2.pml:26 (state 12)
100:	proc  1 (producer:1) prodcons2.pml:20 (state 7)
100:	proc  0 (producer:1) prodcons2.pml:17 (state 12)
4 processes created


In [6]:
# 失败的断言
!spin false.pml

spin: false.pml:2, Error: assertion violated
spin: text of failed assertion: assert(0)
#processes: 1
  1:	proc  0 (:init::1) false.pml:2 (state 1)
1 process created


In [ ]:
# 以验证模式运行
# 生成验证器
!spin -a prodcons2.pml
# 编译验证器
!gcc -w -o pan pan.c
# 执行验证器
!./pan

# 清理
!rm -f pan pan.*

# TODO: 输出的含义


(Spin Version 6.5.1 -- 20 December 2019)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 44 byte, depth reached 5, errors: 0
       10 states, stored
        3 states, matched
       13 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.001	equivalent memory usage for states (stored*(State-vector + overhead))
    0.289	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage


unreached in proctype producer
	prodcons2.pml:25, state 15, "-end-"
	(1 of 15 states)
unreached in proctype consumer
	prodcons2.pml:34, state 15, "-end-"
	(1 of 15 states)

pan: elapsed time 0 seconds


In [13]:
# 互斥问题: 2个进程
!spin -a mutex.pml
!gcc -w -o pan pan.c
!./pan

!rm -f pan pan.*


(Spin Version 6.5.1 -- 20 December 2019)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 28 byte, depth reached 51, errors: 0
      168 states, stored
      156 states, matched
      324 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.009	equivalent memory usage for states (stored*(State-vector + overhead))
    0.291	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage


unreached in proctype mutext
	mutex.pml:37, state 24, "-end-"
	(1 of 24 states)

pan: elapsed time 0 seconds


In [15]:
!rm -f pan pan.*

In [ ]:
# 存在错误的互斥算法
!spin -a mutex_flaw.pml
!gcc -w -o pan pan.c
!./pan

pan:1: assertion violated (cnt==1) (at depth 72)
pan: wrote mutex_flaw.pml.trail

(Spin Version 6.5.1 -- 20 December 2019)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 28 byte, depth reached 72, errors: 1
      261 states, stored
      161 states, matched
      422 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.014	equivalent memory usage for states (stored*(State-vector + overhead))
    0.290	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0.04 seconds
pan: rate      6525 states/second


In [19]:
# 记录每个可达状态的深度
!gcc -w -DREACH -o pan pan.c
# # 指定最大深度
!./pan -i -m72


error: max search depth too small
pan:1: assertion violated (cnt==1) (at depth 61)
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 62
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 61
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 61
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 42
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 41
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 41
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 34
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 33
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 33
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 22
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 21
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 21
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 21
pan: wrote mutex_flaw.pml.trail
pan: reducing search depth to 21

(Spin 

In [20]:
# 使用BFS搜索
!gcc -w -DBFS -o pan pan.c
!./pan

Depth=      10 States=      125 Transitions=      159 Memory=   128.195	
Depth=      20 States=      511 Transitions=      835 Memory=   128.195	
pan:1: assertion violated (cnt==1) (at depth 20)
pan: wrote mutex_flaw.pml.trail

(Spin Version 6.5.1 -- 20 December 2019)
	+ Breadth-First Search
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 28 byte, depth reached 20, errors: 1
      515 states, stored
	     515 nominal states (stored-atomic)
      327 states, matched
      842 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.028	equivalent memory usage for states (stored*(State-vector + overhead))
    0.290	actual memory usage for states
  128.000	memory used for hash table (-w24)
  128.195	total actual memory usage



pan: elapsed time 0.08 seconds


In [22]:
# guided simulation: 重放trail, 观察断言失败
!spin -p -t mutex_flaw.pml

  1:	proc  1 (user:1) mutex_flaw.pml:11 (state 1)	[x = me]
  2:	proc  1 (user:1) mutex_flaw.pml:13 (state 2)	[(((y==0)||(y==me)))]
  3:	proc  1 (user:1) mutex_flaw.pml:13 (state 3)	[(1)]
  4:	proc  1 (user:1) mutex_flaw.pml:17 (state 8)	[z = me]
  5:	proc  1 (user:1) mutex_flaw.pml:19 (state 9)	[((x==me))]
  6:	proc  1 (user:1) mutex_flaw.pml:19 (state 10)	[(1)]
  7:	proc  0 (user:1) mutex_flaw.pml:11 (state 1)	[x = me]
  8:	proc  0 (user:1) mutex_flaw.pml:13 (state 2)	[(((y==0)||(y==me)))]
  9:	proc  0 (user:1) mutex_flaw.pml:13 (state 3)	[(1)]
 10:	proc  1 (user:1) mutex_flaw.pml:23 (state 15)	[y = me]
 11:	proc  1 (user:1) mutex_flaw.pml:25 (state 16)	[((z==me))]
 12:	proc  1 (user:1) mutex_flaw.pml:25 (state 17)	[(1)]
 13:	proc  1 (user:1) mutex_flaw.pml:30 (state 22)	[cnt = (cnt+1)]
 14:	proc  0 (user:1) mutex_flaw.pml:17 (state 8)	[z = me]
 15:	proc  0 (user:1) mutex_flaw.pml:19 (state 9)	[((x==me))]
 16:	proc  0 (user:1) mutex_flaw.pml:19 (state 10)	[(1)]
 17:	proc  0 (user:1) m

In [29]:
# 清理
!rm -f pan pan.* *.trail

zsh:1: no matches found: *.trail


In [27]:
# 互斥问题: Peterson算法 1981: 正确的
!spin -a peterson.pml
!gcc -w -o pan pan.c
!./pan
!rm -f pan pan.* 


(Spin Version 6.5.1 -- 20 December 2019)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 28 byte, depth reached 24, errors: 0
       40 states, stored
       27 states, matched
       67 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.002	equivalent memory usage for states (stored*(State-vector + overhead))
    0.292	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage


unreached in proctype P1
	peterson.pml:24, state 11, "-end-"
	(1 of 11 states)

pan: elapsed time 0 seconds


In [5]:
!rm -f pan pan.* 

In [ ]:
# 消息传递: 没有错误???
# !spin -c protocol.pml

!spin -a protocol.pml
!gcc -w -o pan pan.c
!./pan
# !rm -f pan pan.* 


(Spin Version 6.5.1 -- 20 December 2019)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 44 byte, depth reached 15, errors: 0
       24 states, stored
        6 states, matched
       30 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.002	equivalent memory usage for states (stored*(State-vector + overhead))
    0.289	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage


unreached in proctype Mproc
	(0 of 18 states)
unreached in proctype Wproc
	(0 of 15 states)

pan: elapsed time 0 seconds


# 3. An Overview of PROMELA

对象类型:
- 进程
  - `proctype`, `active [N] proctype`, `run`, `pid`
  - `provided`
- 数据对象
  - `bit`, `bool`, `byte`, `chan`, `mtype`, `pid`, `short`, `int`, `unsigned`
  - 数据结构: `typedef`
- 消息通道
  - `chan`, `!`, `?`, `eval`
  - 预定义函数: `len`, `empty`, `nempty`, `full`, `nfull`
  - 测试消息接收是否可执行: `qname?[msg]`
  - 通道轮询: `qname<msg>` 拷贝消息字段值而不移除消息
  - 有序的发送: `qname!!msg`, 按消息的数值序
  - 随机的接收: `qname??msg`
  - 会合通信(rendezvous communication): `chan port = [0] of {byte}`

语句:
- 打印语句
- 赋值
- IO语句
- 表达式语句

可执行性规则(Rules for Executability)
- 依赖于模型状态, SPIN模型中任意语句是可执行的或阻塞的.
- 将表达式用作语句: 求值为true或非零整数时是可执行的.
  - 例: `(a == b);`
- 表达式必须无副作用, 例外`run`

赋值和表达式
- 没有: `--c`, `++c`
- 有: `c++`, `c--`
- 赋值: `variable = expression`

控制流: 符合语句
- `run sender()`语句
- `skip`表达式: 等价于`(1)`, true.
- 原子序列: `atomic`
- 确定性步骤: `d_step`
- 选择: `if`
- 重复: `do`
- `timeout`语句
- 逃逸序列: `{P} unless {E}`
- 内联定义: `inline`
- C风格的宏: `#define`
- 读取输入: 模拟模式中`chan STDIN`, 消息类型为`int`
- 特殊的特性
  - `xr` 独占接收, `xs` 独占发送
  - `local`: 局部访问变量
  - `hidden`: 不记录状态信息
  - `show`: 变量值变更显示在XSPIN中


More:
- PROMELA语言参考: 16, 17
  - 操作语义: 7 
- 更多的示例: 14, 15

# 4. Defining Correctness Claims

声明的类型
- 状态属性的声明: 状态可达/不可达
  - 系统不变量
  - 进程断言
  - 没有可达的系统死锁状态
- 路径属性的声明: 执行可行/不可行

描述正确性属性的构造 - Summary P.154
- 基本断言
  - `assert(expression)`
  - 唯一可用在模拟运行中的正确性属性类型
- 元标签
  - 结束状态标签: `end`, `end*`
  - 进展状态标签: `progress`, `progress*`
   - `-DNP`
  - 接受状态标签: `accept`, `accept*`
- never声明: 手写, 从逻辑公式或时间线属性描述中生成
- 踪迹断言: `trace`
  - `notrace`


never声明

例: 属性 在每个p为true的系统状态中, 最终会有q为真的系统状态, 在这之间p保持为真.

LTL: ! [] (p -> (p U q))

In [ ]:
# 将LTL公式翻译为never声明
!spin -f '! [] (p -> (p U q))'

# 另一种方式: 时间线编辑器

never  {    /* ! [] (p -> (p U q)) */
T0_init:
	do
	:: (! ((q)) && (p)) -> goto accept_S4
	:: (1) -> goto T0_init
	od;
accept_S4:
	do
	:: (! ((q))) -> goto accept_S4
	:: atomic { (! ((p)) && ! ((q))) -> assert(!(! ((p)) && ! ((q)))) }
	od;
accept_all:
	skip
}


restart here!!!

# 5. Using Design Abstraction

# 6. Automata and Logic

# 7. PROMELA Semantics

操作性语义

# 8. Search Algorithms

# 9. Search Optimization

# 10. Notes on Model Extraction

# 11. Using SPIN

# 12. Notes on XSPIN

# 13. The Timeline Editor

# 14. A Verification Model of a Telephone Switch

# 15. Sample SPIN Models

# 16. PROMELA Language Reference

# 17. Embedded C Code

# 18. Overview of SPIN Options

# 19. Overview of PAN Options

# A. Automata Products

# B. The Great Debates

# C. Exercises With SPIN

# D. Downloading Spin